In [1]:
from keras.applications import InceptionV3

In [2]:
IMG_SIZE = 128

# model = InceptionV3(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     name="inception_v3",
# )

model = InceptionV3(include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

In [3]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 63, 63, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 63, 63, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

feat_extractor = Sequential(name="Custom Inception v3 Feature Extractor")
# for i, layer in enumerate(model.layers):
#     if (i != len(model.layers) - 1):
#         feat_extractor.add(layer)

# Assume `model` is your pre-loaded Functional model
feat_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
feat_extractor.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 63, 63, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 63, 63, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [5]:
from keras.utils import plot_model
plot_model(feat_extractor, to_file='feature_extractor_inception_v3.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [6]:
# Test forward pass
import numpy as np
subimage_data = np.load('../data_collection/data/img_data_20241030_0130.npy')
gesture_seq   = subimage_data[0]
print(gesture_seq.shape)

#output = np.zeros((10, 1000))
output = np.zeros((10, 2048))

for i in range(gesture_seq.shape[3]):
    subimg = np.expand_dims(gesture_seq[:, :, :, i], 0)
    #output[i] = model.predict(subimg)
    output[i] = feat_extractor.predict(subimg)

print(output.shape)
print(output[0, :20])

(128, 128, 3, 10)
1/1 [==============================] - 0s 35ms/step
(10, 2048)
[0.29266018 0.56488103 0.         0.         0.23348483 0.
 0.15342468 0.         0.4476555  0.32609361 0.19165552 0.
 0.14391781 0.04680949 0.28743988 0.         0.00750713 0.22647467
 0.05954104 0.        ]
